In [ ]:
import numpy as np
import mne

from hdf5storage import loadmat, savemat 
from mne.minimum_norm import make_inverse_operator, apply_inverse, compute_source_psd_epochs, write_inverse_operator

from matplotlib import pyplot as plt


In [ ]:
outdict=loadmat('preprocessed_eeg.mat')
outdict.keys()

In [ ]:
preprocessed_eeg=outdict['preprocessed_eeg']*0.000001 # reduce the amplitube to be shown on MNE's plot
sampling_freq=outdict['Fs'][0]


In [ ]:
# preprocessed_eeg[1,1000:1100]

In [ ]:
# data=raw.get_data()

In [ ]:
# data[1,1000:1100]

In [ ]:
ch_bad=[] # ch_bad=outdict['ch_bad'][0]-1
ch_dubious=[] # ch_dubious=outdict['ch_dubious'][0]-1

In [ ]:

subject_ID=outdict['subject_ID'][0]

n_channels=np.shape(preprocessed_eeg)[0]
ch_info_bads=np.concatenate((ch_bad, ch_dubious), axis=0)
ch_info_bads=np.unique(ch_info_bads)


In [ ]:
ch_labels=outdict['ch_labels'].tolist()

In [ ]:
ch_labels_names=list()
for i in range(32):
    ch_labels_names.append(ch_labels[i][0][0])

In [ ]:
ch_names = [f"E{n}" for n in range(1, 33)]


In [ ]:
ch_types = ["eeg"] * n_channels
info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
info.set_montage("GSN-HydroCel-32",match_case=False,match_alias=False,on_missing='raise', verbose=None)
info["description"] = subject_ID
print(info)

In [ ]:
# info.keys()

In [ ]:
# info['chs']

In [ ]:
# alternative method 
# raw=mne.io.read_raw_fif('./TMSi32chan_loc_small_example.fif') # (this fif file is imported from original Poly5 to EDF to MNE)
# missing dig in the info

In [ ]:
# len(info['dig'])

In [ ]:
outdict=loadmat('./xyzlabels.mat') # (this xyzlabels.mat has coordinates from original Poly to EDF to MNE)
x=outdict['x']
y=outdict['y']
z=outdict['z']

# this set of xyz does not look right to work as digitaization points for coregistartion, but works for topoplot
for ch in range(3,len(info['dig'])):
    info['dig'][ch]['r'][0]=x[ch-3]
    info['dig'][ch]['r'][1]=y[ch-3]
    info['dig'][ch]['r'][2]=z[ch-3]

In [ ]:
# info['dig']

In [ ]:
# info['chs'][0] # don't know why it is different from dig

In [ ]:
# outdict=loadmat('./xyzlabels.mat') # (this xyzlabels.mat has coordinates from original Poly to EDF to MNE)
# x=outdict['x']
# y=outdict['y']
# z=outdict['z']

for ch in range(32):
    info['chs'][ch]['loc'][0]=x[ch]
    info['chs'][ch]['loc'][1]=y[ch]
    info['chs'][ch]['loc'][2]=z[ch]

In [ ]:
# info['dig']

In [ ]:
# info['chs'][0]

In [ ]:
outdict=loadmat('./xyzlabels.mat') # (this xyzlabels.mat has coordinates from original Poly to EDF to MNE)
Coordinates=np.zeros((32,3))
Coordinates[:,0]=outdict['x']
Coordinates[:,1]=outdict['y']
Coordinates[:,2]=outdict['z']

In [ ]:
# Coordinates=outdict['Coordianates']/1000
# for ch in range(3,len(info['dig'])):
#     info['dig'][ch]['r']=Coordinates[ch-3,:]

In [ ]:
# mark the bad channels
ch_labels_info_bads=list()
for k in range(len(ch_info_bads)):
    ch_labels_info_bads.append(ch_names[ch_info_bads[k]])

info['bads'] = ch_labels_info_bads

In [ ]:
raw = mne.io.RawArray(preprocessed_eeg, info)
raw.set_eeg_reference('average', projection=True)

In [ ]:
# raw.plot(show_scrollbars=False, show_scalebars=False)

In [ ]:
raw.save("raw.fif",overwrite=True)

In [ ]:
# fig = mne.viz.plot_alignment(
#     raw.info)
# mne.viz.set_3d_view(fig)

In [ ]:
# # topoplot
# import numpy as np
# x=np.random.rand(32)
# x=list(x*100)
# # x=list(range(1,33,1))
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(1)
# mne.viz.plot_topomap(x, \
#     raw.info, vlim=(None, None), axes=ax,\
#         sensors=True, names=ch_labels_names, cmap='seismic')

coregistration

In [ ]:
# mne.gui.coregistration()

forward solution

In [ ]:
# the raw file containing the channel location + types
raw_fname = './raw.fif'

# The paths to Freesurfer reconstructions
subjects_dir ='../../archive/subjects/'
subject = 'fsaverage' # change it to use freesurfer's bem

# Compute Source Space (surface)
ico = 4 #**************************************************************
spacing='ico'+str(ico) 
src = mne.setup_source_space(subject, spacing=spacing, add_dist='patch',
                             subjects_dir=subjects_dir)
conductivity = (0.3, 0.0075, 0.3)  #  three layers for EEG (MNE default  (0.3 0.006 0.3) )
model = mne.make_bem_model(subject=subject, ico=ico,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
# 5 s


In [ ]:
bem = mne.make_bem_solution(model)

trans = './tmsi_trans.fif'
# 1m 10s

In [ ]:
# # view surfaces
# plot_bem_kwargs = dict(
#     subject=subject,
#     subjects_dir=subjects_dir,
#     brain_surfaces="white",
#     orientation="coronal",
#     slices=[50, 100, 150, 200],
# )

# mne.viz.plot_bem(**plot_bem_kwargs)

In [ ]:
# # visualing the coregistration 
# info = mne.io.read_info(raw_fname)
# # Here we look at the dense head, which isn't used for BEM computations but
# # is useful for coregistration.
# mne.viz.plot_alignment(
#     info,
#     trans,
#     subject=subject,
#     dig=True,
#     subjects_dir=subjects_dir,
#     surfaces="pial",
# )

In [ ]:
# mne.viz.plot_bem(src=src, **plot_bem_kwargs)

In [ ]:
# surface = '../../archive/subjects/fsaverage/bem/inner_skull.surf'
# vol_src = mne.setup_volume_source_space(
#     subject, subjects_dir=subjects_dir, surface=surface, add_interpolator=False
# )  # Just for speed!
# print(vol_src)

# mne.viz.plot_bem(src=vol_src, **plot_bem_kwargs)

In [ ]:
# fig = mne.viz.plot_alignment(
#     subject=subject,
#     subjects_dir=subjects_dir,
#     surfaces="white",
#     coord_frame="mri",
#     src=src,
# )
# mne.viz.set_3d_view(
#     fig,
#     azimuth=173.78,
#     elevation=101.75,
#     distance=0.30,
#     focalpoint=(-0.03, -0.01, 0.03),
# )

In [ ]:
fwd = mne.make_forward_solution(raw_fname, trans=trans, src=src, bem=bem,
                                meg=False, eeg=True, mindist=5.0, n_jobs=2,
                                verbose=True)
print(fwd)

# 10s


In [ ]:
# lh = fwd["src"][0]  # Visualize the left hemisphere
# verts = lh["rr"]  # The vertices of the source space
# tris = lh["tris"]  # Groups of three vertices that form triangles
# dip_pos = lh["rr"][lh["vertno"]]  # The position of the dipoles
# dip_ori = lh["nn"][lh["vertno"]]
# dip_len = len(dip_pos)
# dip_times = [0]
# white = (1.0, 1.0, 1.0)  # RGB values for a white color

# actual_amp = np.ones(dip_len)  # misc amp to create Dipole instance
# actual_gof = np.ones(dip_len)  # misc GOF to create Dipole instance
# dipoles = mne.Dipole(dip_times, dip_pos, actual_amp, dip_ori, actual_gof)
# trans = mne.read_trans('tmsi_trans.fif')

# fig = mne.viz.create_3d_figure(size=(600, 400), bgcolor=white)
# coord_frame = "mri"

# # Plot the cortex
# mne.viz.plot_alignment(
#     subject=subject,
#     subjects_dir=subjects_dir,
#     trans=trans,
#     surfaces="white",
#     coord_frame=coord_frame,
#     fig=fig,
# )

# # Mark the position of the dipoles with small red dots
# mne.viz.plot_dipole_locations(
#     dipoles=dipoles,
#     trans=trans,
#     mode="sphere",
#     subject=subject,
#     subjects_dir=subjects_dir,
#     coord_frame=coord_frame,
#     scale=7e-4,
#     fig=fig,
# )

# mne.viz.set_3d_view(figure=fig, azimuth=180, distance=0.25)

In [ ]:
# fig = mne.viz.create_3d_figure(size=(600, 400))

# # Plot the cortex
# mne.viz.plot_alignment(
#     subject=subject,
#     subjects_dir=subjects_dir,
#     trans=trans,
#     surfaces="white",
#     coord_frame="head",
#     fig=fig,
# )

# # Show the three dipoles defined at each location in the source space
# mne.viz.plot_alignment(
#     subject=subject,
#     subjects_dir=subjects_dir,
#     trans=trans,
#     fwd=fwd,
#     surfaces="white",
#     coord_frame="head",
#     fig=fig,
# )

# mne.viz.set_3d_view(figure=fig, azimuth=180, distance=0.1)

In [ ]:
# fig = mne.viz.create_3d_figure(size=(600, 400))

# # Plot the cortex
# mne.viz.plot_alignment(
#     subject=subject,
#     subjects_dir=subjects_dir,
#     trans=trans,
#     surfaces="white",
#     coord_frame="head",
#     fig=fig,
# )

# # Show the dipoles as arrows pointing along the surface normal
# mne.viz.plot_dipole_locations(
#     dipoles=dipoles,
#     trans=trans,
#     mode="arrow",
#     subject=subject,
#     subjects_dir=subjects_dir,
#     coord_frame="head",
#     scale=7e-4,
#     fig=fig,
# )

# mne.viz.set_3d_view(figure=fig, azimuth=180, distance=0.1)

In [ ]:
# src

In [ ]:
fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True)
leadfield = fwd_fixed['sol']['data']
source_rr=fwd_fixed['source_rr']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

In [ ]:
mne.write_forward_solution('fwd.fif', fwd_fixed, overwrite=True, verbose=None)

inv

In [ ]:
# raw=mne.io.read_raw_fif(raw_fname)

In [ ]:
# use the entire data as empty room noise
noise_cov = mne.compute_raw_covariance(raw)
noise_cov_mat=noise_cov['data'] # it only included the good chans
# compute the average of the diagonal
# construct the scale of the noise cov
scale=0.05 # 1% 5% 10% 30% 50% *******************************************************************************
scale_ave=np.sqrt(np.mean(noise_cov_mat.diagonal()))*scale
sim_cov_mat=np.zeros((np.shape(noise_cov_mat)))
for i in range(np.shape(sim_cov_mat)[0]):
       sim_cov_mat[i,i]=scale_ave


In [ ]:
# noise_cov_mat.diagonal()

In [ ]:
# plt.imshow(noise_cov_mat,vmin=-1.5e-10, vmax=1.5e-10, cmap='jet')
# plt.colorbar()
# np.mean(noise_cov_mat.diagonal())

In [ ]:
sim_cov=noise_cov.copy()
sim_cov['data']=sim_cov_mat


In [ ]:
# mne.write_cov('noise_cov.fif', noise_cov, overwrite=True, verbose=None)
# mne.write_cov('sim_cov.fif', sim_cov, overwrite=True, verbose=None)

In [ ]:
# nepochs=int(np.floor(np.shape(preprocessed_eeg)[1]/sampling_freq))
# epochs_mat = np.column_stack(
#     (
#         np.arange(0, np.shape(preprocessed_eeg)[1]-sampling_freq, sampling_freq),
#         np.zeros(nepochs, dtype=int),
#         np.array([1]*nepochs),
#     )
# )


In [ ]:

# epoch_dict = dict(resting=1)
# epochs = mne.Epochs(raw, epochs_mat, tmin=0, tmax=0.999, event_id=epoch_dict, preload=True, baseline=(None, None))

# evoked = epochs["resting"].average()
# mne.write_evokeds('resting_ave.fif', evoked, on_mismatch='raise', overwrite=True, verbose=None)
# evokeds = mne.read_evokeds('./resting_ave.fif')
# resting = evokeds[0]

In [ ]:
# fwd = mne.read_forward_solution('fwd.fif')
fwd_surf=mne.convert_forward_solution(fwd, surf_ori=True, copy=False)

In [ ]:
# Compute the source estimate (Fixed dipole orientations)
depth=0.8 # 0.8 1 2 4 ***************************************************************************************
# inverse_operator = make_inverse_operator(resting.info, fwd, sim_cov, depth=depth, fixed=True)
inverse_operator = make_inverse_operator(raw.info, fwd_surf, sim_cov, depth=depth, fixed=True)
# save the inverse operator and the inverse matrix
# write_inverse_operator('inv.fif', inverse_operator,overwrite=True)

linearly constrained minimum variance (LCMV) beamformer

In [1]:
# from mne.beamformer import make_lcmv, apply_lcmv, apply_lcmv_raw

In [ ]:
# filters = make_lcmv(
#     raw.info,
#     fwd,
#     noise_cov,
#     reg=0.05,
#     noise_cov=noise_cov,
#     pick_ori="max-power",
#     weight_norm="unit-noise-gain",
#     rank=None,
# )

# # You can save the filter for later use with:
# # filters.save('filters-lcmv.h5')

# # stc = apply_lcmv(evoked, filters)
# stc = apply_lcmv_raw(raw,filters) 
# # 32s

In [ ]:
# np.shape(stc.data)

In [ ]:
# filters_vec = make_lcmv(
#     evoked.info,
#     fwd,
#     noise_cov,
#     reg=0.05,
#     noise_cov=sim_cov,
#     pick_ori="vector",
#     weight_norm="unit-noise-gain-invariant",
#     rank=None,
# )
# # save a bit of memory
# # src = fwd_fixed["src"]
# # del fwd_fixed
# stc_vec = apply_lcmv(evoked, filters_vec)

In [ ]:
%run extract_invmat.ipynb # run this script to load the functions needed to extract inverse matrix

In [ ]:
methods = ['MNE','dSPM','sLORETA','eLORETA'] # **********************************************
method = methods[0]  # (could also be dSPM or sLORETA)# **********************************************
# invmat=_get_matrix_from_inverse_operator(inverse_operator,fwd,method=method,lambda2=1.0/9.0)
invmat=_get_matrix_from_inverse_operator(inverse_operator,fwd_fixed,method=method,lambda2=1.0/9.0)
np.shape(invmat)

Correlation

In [ ]:
# not ch_info_bads

In [ ]:
# remove bad channels from the original eeg
if ch_info_bads:
    original_eeg=np.delete(preprocessed_eeg,ch_info_bads,0)
else:
    original_eeg=preprocessed_eeg

In [ ]:
# reconstruct eeg using invmat and leadfield matrix
source_data=np.matmul(invmat,original_eeg)
EEG_recon=np.matmul(leadfield,source_data)
# 11s

In [ ]:
# remove bad channels from the reconstructed eeg
if ch_info_bads:
    EEG_recon=np.delete(EEG_recon,ch_info_bads,0)


In [ ]:
# plt.plot(np.arange(0,np.shape(EEG_recon)[1],1),EEG_recon.transpose())
# plt.title('reconstructed EEG - all chan')
# plt.ylim([-2e-5,2e-5])
# # 16 s

In [ ]:
# plt.plot(np.arange(0,np.shape(EEG_recon)[1],1),EEG_recon[ch_dubious,:].transpose())
# plt.title('reconstructed EEG - dubious chan')
# plt.ylim([-2e-5,2e-5])

In [ ]:
# plt.plot(np.arange(0,np.shape(EEG_recon)[1],1),EEG_recon[ch_bad,:].transpose())
# plt.title('reconstructed EEG - bad chan')
# plt.ylim([-2e-5,2e-5])

In [ ]:
# compute correlation
corr = np.corrcoef(EEG_recon, original_eeg, rowvar=True)

In [ ]:
# # examine 
# chan=22 # 12, 30
# plt.subplot(1,2,1)
# plt.plot(np.arange(np.size(original_eeg,1)),original_eeg[chan,:],'r')
# plt.title('original_eeg')
# plt.subplot(1,2,2)
# plt.plot(np.arange(np.size(EEG_recon,1)),EEG_recon[chan,:],'b')
# plt.title('reconstructed eeg')

In [ ]:
# # examine
# chan=31 # 12, 30
# nsamples=2000
# plt.subplot(1,2,1)
# plt.plot(np.arange(nsamples),original_eeg[chan,10000:10000+nsamples],'r')
# plt.title('original_eeg')
# plt.subplot(1,2,2)
# plt.plot(np.arange(nsamples),EEG_recon[chan,10000:10000+nsamples],'b')
# plt.title('reconstructed eeg')

In [ ]:
# corr

In [ ]:
# np.arange(0,np.shape(EEG_recon)[0],1)+194

In [ ]:
# corr=corr[np.arange(0,np.shape(EEG_recon)[0],1)]

In [ ]:
# np.shape(corr)

In [ ]:
# corr=corr[:,np.arange(194,388,1)]

In [ ]:
# plt.imshow(corr,cmap='jet',vmin=-1,vmax=1)
# plt.ylabel('reconstructed EEG (194 good channels)')
# plt.xlabel('original EEG (194 good channels)')
# plt.title('correlation')
# plt.colorbar()

In [ ]:
# plt.imshow(corr,cmap='jet',vmin=-1,vmax=1)
# plt.colorbar()
# plt.title('corrceof: ico' + str(ico) + ' scale ' +str(scale) +' depth '+ str(depth))

In [ ]:
corrcoef_diag=np.zeros((np.shape(EEG_recon)[0]))
for i in range(np.shape(EEG_recon)[0]):
    corrcoef_diag[i]=corr[i,np.shape(EEG_recon)[0]+i]


In [ ]:
corrcoef_diag

In [ ]:
# plt.plot(np.arange(0, np.shape(EEG_recon)[0],1) ,corrcoef_diag, 'r.')
# plt.title('corrceof: ico' + str(ico) + ' scale ' +str(scale) +' depth '+ str(depth))
# plt.xlabel('good channels')
# plt.ylabel('corrcoef')

In [ ]:
# corrcoef_diag # chan 13 (M1) and chan 31(OZ) have lowest correlation

In [ ]:
# # save outputs all in one mat file. 
# outdict=dict()
# outdict['leadfield']=leadfield # leadfield matrix
# outdict['invmat']=invmat #  inverse matrix
# outdict['source_rr']=source_rr # the source location AND labels (matlab)
# outdict['sim_cov_mat']=sim_cov_mat # noise covariance
# outdict['scale']=scale # scale
# outdict['depth']=depth # depth
# outdict['conductivity']=conductivity # 3 layrs conductivity


# outdict['ch_bad']=ch_bad #  inverse matrix
# outdict['ch_dubious']=ch_dubious # 
# outdict['ch_names']=ch_names #  
# outdict['Coordinates']=Coordinates #  
# outdict['corrcoef_diag']=corrcoef_diag #  
# outdict['subject_ID']=subject_ID #  

# savemat(subject_ID+'_scale_'+str(scale)+'_depth_'+str(depth),outdict)

In [ ]:
outdict=dict()
outdict['leadfield']=leadfield # leadfield matrix
outdict['invmat']=invmat #  inverse matrix
outdict['source_rr']=source_rr # the source location AND labels (matlab)
outdict['sim_cov_mat']=sim_cov_mat # noise covariance
# stc_data=stc.data
# outdict['stc']=stc_data # the source estimate
outdict['scale']=scale # scale
outdict['depth']=depth # depth
outdict['conductivity']=conductivity # 3 layrs conductivity
outdict['ch_bad']=ch_bad #  inverse matrix
outdict['ch_dubious']=ch_dubious # 
outdict['ch_names']=ch_names #  
outdict['Coordinates']=Coordinates #  
outdict['corrcoef_diag']=corrcoef_diag #  
outdict['subject_ID']=subject_ID #  
savemat(subject_ID +'_method_' + methods[0]+'_ico_'+str(ico)+'_scale_'+str(scale)+'_depth_'+str(depth),outdict)


In [ ]:
np.shape(stc.data)

In [ ]:
# outdict=dict()
stc_data=stc.data
stc_data1=stc_data[0:1000,:]
stc_data2=stc_data[1000:2000,:]
stc_data3=stc_data[2000:3000,:]
stc_data4=stc_data[3000:4000,:]
stc_data5=stc_data[4000::,:]

In [ ]:
# stc.save("stc.fif",overwrite=True, ftype='h5')

In [ ]:
# outdict['stc_data']=stc_data # the source estimate


In [ ]:
# import scipy
# scipy.__version__


In [ ]:
# import sklearn
# sklearn.__version__

In [ ]:
# pip install scipy==1.11.3
# the previous version is 1.9 compatible with sklearn 1.2.0

In [ ]:
# scipy.io.savemat(subject_ID +'_method_' + methods[0]+'_ico_'+str(ico)+'_scale_'+str(scale)+'_depth_'+str(depth)+'_st.data',
#                  outdict,oned_as='column',format='5',do_compression=True)
# # 1 min

In [ ]:
# import pickle as pkl
# filename = subject_ID +'_method_' + methods[0]+'_ico_'+str(ico)+'_scale_'+str(scale)+'_depth_'+str(depth)+'_st.data'
# fileObject = open(filename, 'wb')
# pkl.dump(stc_data, fileObject)
# fileObject.close()

In [ ]:
# import hdf5storage
# hdf5storage.__version__
# # it was 0.1.18 previously

In [ ]:
# pip install hdf5storage==0.1.19

In [ ]:
# savemat(subject_ID +'_method_' + methods[0]+'_ico_'+str(ico)+'_scale_'+str(scale)+'_depth_'+str(depth)+'_st.data.mat',
#         outdict,do_compression=True,format='7.3')

In [ ]:
savemat(subject_ID +'_method_' + methods[0]+'_ico_'+str(ico)+'_scale_'+str(scale)+'_depth_'+str(depth)+'_st.data1.mat',
        {"stc_data1": stc_data1},do_compression=True,format='7.3')


In [ ]:
savemat(subject_ID +'_method_' + methods[0]+'_ico_'+str(ico)+'_scale_'+str(scale)+'_depth_'+str(depth)+'_st.data2.mat',
        {"stc_data2": stc_data2},do_compression=True,format='7.3')
savemat(subject_ID +'_method_' + methods[0]+'_ico_'+str(ico)+'_scale_'+str(scale)+'_depth_'+str(depth)+'_st.data3.mat',
        {"stc_data3": stc_data3},do_compression=True,format='7.3')
savemat(subject_ID +'_method_' + methods[0]+'_ico_'+str(ico)+'_scale_'+str(scale)+'_depth_'+str(depth)+'_st.data4.mat',
        {"stc_data4": stc_data4},do_compression=True,format='7.3')
savemat(subject_ID +'_method_' + methods[0]+'_ico_'+str(ico)+'_scale_'+str(scale)+'_depth_'+str(depth)+'_st.data5.mat',
        {"stc_data5": stc_data5},do_compression=True,format='7.3')